In [ ]:
!pip install llama_cpp_python
!pip install pytelegrambotapi

In [ ]:
import json
import sqlite3
from llama_cpp import Llama
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.5-GGUF",
    filename="functionary-small-v2.5.Q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.5-GGUF"),
    n_gpu_layers=-1
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--meetkai--functionary-small-v2.5-GGUF/snapshots/37e7f0678f49a1d360090d0f4b79d138ca2248ee/./functionary-small-v2.5.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = llama3-functionary-hf
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_lo

Funções da aplicação

In [ ]:
import sqlite3

global pedido
pedido = {'Cliente': '', 'itens': [], 'valor': '', 'status': 'In progress', 'forma_pagamento': ''}

def calcular_total():
    valor = 0
    for item in pedido['itens']:
        valor += item['valor']
    pedido['valor'] = valor

#Dar função itens_disponiveis() no enum
def adicionar_item(item):
    conexao = sqlite3.connect('cardapio.db')
    cursor = conexao.cursor()
    cursor.execute('SELECT * FROM cardapio WHERE nome = ?', (item,))
    item_info = cursor.fetchone()
    if item_info:
        novo_item = {'nome': item_info[1], 'tipo': item_info[2], 'valor': item_info[3]}
        pedido['itens'].append(novo_item)
    conexao.close()
    return f"Great! I just added {item} to your order!"

#Dar função itens_pedido() no enum
def remover_item(item):
    for i in range(len(pedido['itens'])):
        if pedido['itens'][i]['nome'] == item:
            pedido['itens'].pop(i)
            break
    return f"Ok! I just removed {item} from your order!"

def calcular_valor():
    calcular_total()
    return f"The total value of your order is ${valor}"

def finalizar_pedido():
    calcular_total()
    if pedido['Cliente'] != '' and pedido['forma_pagamento'] != '' and pedido['itens'] != []:
        pedido['status'] = 'Finalized'
        return f"Order finalized! I just sent it to our kitchen team. Thank you for ordering with us!"
    return f"Oops! It seems like some information is missing to finalize the order. Please check if you provided your name, payment method, and the items in the order."

def cancelar_pedido():
    pedido['status'] = 'Cancelled'
    return f"Order cancelled! If you need anything else, just let me know!"

#Dar Array com formas de pagamento
def forma_pagamento(forma):
    pedido['forma_pagamento'] = forma
    return f"Chosen payment method: {forma}"

def informacoes_pedido():
    calcular_total()
    return f"Here are the details of your order so far:\nCustomer: {pedido['Cliente']}\nItems: {itens_pedido_print()}\nValue: {pedido['valor']}\nStatus: {pedido['status']}\nPayment Method: {pedido['forma_pagamento']}"

def itens_pedido():
    itens = []
    for item in pedido['itens']:
        itens.append(item['nome'])
    return itens

def itens_pedido_print():
    resp = 'Items in your order:\n'
    for item in pedido['itens']:
        resp = resp + f"Name: {item['nome']}\nType: {item['tipo']}\nPrice: {item['valor']}\n"
    return resp

def itens_disponiveis():
    conexao = sqlite3.connect('cardapio.db')
    cursor = conexao.cursor()
    cursor.execute('SELECT * FROM cardapio WHERE disponibilidade = 1')
    itens = cursor.fetchall()
    disponiveis = []
    for i in itens:
        disponiveis.append(i[1])
    conexao.close()
    return disponiveis

def itens_disponiveis_print():
    resp = 'This is our menu:\n'
    conexao = sqlite3.connect('cardapio.db')
    cursor = conexao.cursor()
    cursor.execute('SELECT * FROM cardapio WHERE disponibilidade = 1')
    itens = cursor.fetchall()
    for item in itens:
        resp = resp + f"Name: {item[1]}\nPrice: {item[3]}\nType: {item[2]}\n"
    conexao.close()
    return resp

def adicionar_cliente(nome):
    pedido['Cliente'] = nome
    return f"Great! {nome}, I just added your name to the order!"

def iniciar_pedido():
    pedido = {'Cliente': '', 'itens': [], 'valor': '', 'status': 'In progress', 'forma_pagamento': ''}
    return '''Order started!
    To finalize your order, I will need some information.
    Your name, what you want to order, and the payment method.
    '''

def iniciar():
    return '''Welcome to the ordering system!
    How can I assist you?
    Ask me what delicious items are on our menu!
    Tell me what you would like to order!
    I can help you with your order!
    '''

In [ ]:
#------------------------------------Funções Para o telegram-------------------------------------------------------

def question(s):
  messages[0]["content"] = s

def response():
  try:
    result = llm.create_chat_completion(
        messages = messages,
        tools=tools,
        tool_choice="auto",
    )
  except:
    print('Error')
    return 'Sorry, it seems an error has occurred, please try again later.'
  #s = result["choices"][0]["message"]["function_call"]

  print(f'Print: {result}')
  s = result['choices'][0]['message']['tool_calls'][0]['function']['name']


  content = s.split('functions.')[1].split('\n')[0]
  if '{}' in s:
    functionCall = f"{content}()"
    print(functionCall)
    return eval(functionCall)
  value = s.split(': "')[1].split('"}')[0]
  functionCall = f"{content}('{value}')"
  print(functionCall)
  return eval(functionCall)

Banco de Dados Exemplo

In [ ]:
# Conectar-se ao banco de dados (se não existir, será criado automaticamente)
conexao = sqlite3.connect('cardapio.db')

# Criar um cursor
cursor = conexao.cursor()

# Criar uma tabela
cursor.execute('''CREATE TABLE IF NOT EXISTS cardapio (
    id INTEGER PRIMARY KEY,
    nome TEXT,
    tipo TEXT,
    preco FLOAT,
    disponibilidade BOOLEAN
    )''')

# Inserir dados na tabela
menu_items = [
    ('Classic Burger', 'Burger', 8.99, 1),
    ('Cheeseburger', 'Burger', 9.99, 1),
    ('Mushroom Swiss Burger', 'Burger', 10.49, 0),
    ('Veggie Burger', 'Burger', 9.49, 1),
    ('Turkey Burger', 'Burger', 9.99, 0),
    ('Chicken Sandwich', 'Sandwich', 9.99, 1),
    ('Onion Rings', 'Side', 4.99, 1),
    ('Coke', 'Drink', 2.99, 1),
    ('Sprite', 'Drink', 2.99, 0),
    ('Iced Tea', 'Drink', 2.99, 1),
    ('French Fries', 'Side', 3.99, 1),
]

# Insert data using executemany
cursor.executemany("INSERT INTO cardapio (nome, tipo, preco, disponibilidade) VALUES (?, ?, ?, ?)", menu_items)


# Executar uma consulta SQL
cursor.execute("SELECT * FROM cardapio")
print(cursor.fetchall())  # Exibir todos os resultados

# Commit e fechar a conexão
conexao.commit()
conexao.close()

[(1, 'Classic Burger', 'Burger', 8.99, 1), (2, 'Cheeseburger', 'Burger', 9.99, 1), (3, 'Mushroom Swiss Burger', 'Burger', 10.49, 0), (4, 'Veggie Burger', 'Burger', 9.49, 1), (5, 'Turkey Burger', 'Burger', 9.99, 0), (6, 'Chicken Sandwich', 'Sandwich', 9.99, 1), (7, 'Onion Rings', 'Side', 4.99, 1), (8, 'Coke', 'Drink', 2.99, 1), (9, 'Sprite', 'Drink', 2.99, 0), (10, 'Iced Tea', 'Drink', 2.99, 1), (11, 'French Fries', 'Side', 3.99, 1)]


In [ ]:
conexao = sqlite3.connect('cardapio.db')

# Criar um cursor
cursor = conexao.cursor()

cursor.execute('DROP TABLE cardapio')
conexao.commit()
conexao.close()

In [ ]:
# Conectar-se ao banco de dados (se não existir, será criado automaticamente)
conexao = sqlite3.connect('cardapio.db')

# Criar um cursor
cursor = conexao.cursor()

# Criar uma tabela
cursor.execute('''CREATE TABLE IF NOT EXISTS cardapio (
    id INTEGER PRIMARY KEY,
    nome TEXT,
    tipo TEXT,
    preco FLOAT,
    disponibilidade BOOLEAN
    )''')

# Inserir dados na tabela
menu_items = [
    ('Classic Burger', 'Burger', 8.99, 1),
    ('Cheeseburger', 'Burger', 9.99, 1),
    ('Mushroom Swiss Burger', 'Burger', 10.49, 0),
    ('Veggie Burger', 'Burger', 9.49, 1),
    ('Turkey Burger', 'Burger', 9.99, 0),
    ('Chicken Sandwich', 'Sandwich', 9.99, 1),
    ('Onion Rings', 'Side', 4.99, 1),
    ('Coke', 'Drink', 2.99, 1),
    ('Sprite', 'Drink', 2.99, 0),
    ('Iced Tea', 'Drink', 2.99, 1),
    ('French Fries', 'Side', 3.99, 1),
]

# Insert data using executemany
cursor.executemany("INSERT INTO cardapio (nome, tipo, preco, disponibilidade) VALUES (?, ?, ?, ?)", menu_items)


# Executar uma consulta SQL
cursor.execute("SELECT * FROM cardapio")
print(cursor.fetchall())  # Exibir todos os resultados

# Commit e fechar a conexão
conexao.commit()
conexao.close()

[(1, 'Classic Burger', 'Burger', 8.99, 1), (2, 'Cheeseburger', 'Burger', 9.99, 1), (3, 'Mushroom Swiss Burger', 'Burger', 10.49, 0), (4, 'Veggie Burger', 'Burger', 9.49, 1), (5, 'Turkey Burger', 'Burger', 9.99, 0), (6, 'Chicken Sandwich', 'Sandwich', 9.99, 1), (7, 'Onion Rings', 'Side', 4.99, 1), (8, 'Coke', 'Drink', 2.99, 1), (9, 'Sprite', 'Drink', 2.99, 0), (10, 'Iced Tea', 'Drink', 2.99, 1), (11, 'French Fries', 'Side', 3.99, 1)]


fim

In [ ]:
messages = [
    {"role": "user", "content": "what is 75 minus 13?"}
]

tools = [
    #Função adicionar_item(item)
    {
        "type": "function",
        "function": {
            "name": "adicionar_item",
            "description": "Add item to the current order.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item": {
                        "type": "string",
                        "description": "The name of the item that will be added to the order.",
                        "enum": itens_disponiveis()
                    }
                },
                "required": ["item"]
            }
        }
    },
    #Função remover_item(item)
    {
        "type": "function",
        "function": {
            "name": "remover_item",
            "description": "Remove item from the current order.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item": {
                        "type": "string",
                        "description": "The name of the item that will be removed from the order.",
                        "enum": itens_pedido()
                    }
                },
                "required": ["item"]
            }
        }
    },
    #Função calcular_valor()
    {
        "type": "function",
        "function": {
            "name": "calcular_valor",
            "description": "Calculate the total value of the current order and return the result.",
            "parameters": {
                "type": "object",
            }
        }
    },
    #Função finalizar_pedido()
    {
        "type": "function",
        "function": {
            "name": "finalizar_pedido",
            "description": "Finalize the current order and send it to the kitchen team.",
            "parameters": {
            }
        }
    },
    #Função cancelar_pedido()
    {
        "type": "function",
        "function": {
            "name": "cancelar_pedido",
            "description": "Cancel the current order.",
            "parameters": {
                "type": "object",
            }
        }
    },
    #Função forma_pagamento(forma)
    {
        "type": "function",
        "function": {
            "name": "forma_pagamento",
            "description": "Choose the payment method for the current order.",
            "parameters": {
                "type": "object",
                "properties": {
                    "forma": {
                        "type": "string",
                        "description": "The payment method chosen by the customer.",
                        "enum": ['Credit Card', 'Debit Card', 'Cash']
                    }
                },
                "required": ["forma"]
            }
        }
    },
    #Função informacoes_pedido()
    {
        "type": "function",
        "function": {
            "name": "informacoes_pedido",
            "description": "Return the details of the current order.",
            "parameters": {
                "type": "object",
            }
        }
    },
    #Função itens_pedido_print()
    {
        "type": "function",
        "function": {
            "name": "itens_pedido_print",
            "description": "Return the items in the current order.",
            "parameters": {
                "type": "object",
            }
        }
    },
    #Função itens_disponiveis_print()
    {
        "type": "function",
        "function": {
            "name": "itens_disponiveis_print",
            "description": "Return the items available in the menu.",
            "parameters": {
                "type": "object",
            }
        }
    },
    #Função adicionar_cliente(nome)
    {
        "type": "function",
        "function": {
            "name": "adicionar_cliente",
            "description": "Add the customer's name to the current order.",
            "parameters": {
                "type": "object",
                "properties": {
                    "nome": {
                        "type": "string",
                        "description": "The name of the customer"
                    }
                },
                "required": ["nome"]
            }
        }
    },
    #Função iniciar_pedido()
    {
        "type": "function",
        "function": {
            "name": "iniciar_pedido",
            "description": "Start a new order.",
            "parameters": {
                "type": "object",
            }
        }
    },
]

In [ ]:
import telebot

In [ ]:
API_TOKEN = 'You API bot Token'
bot = telebot.TeleBot(API_TOKEN)

In [ ]:
@bot.message_handler(func=lambda message: True, content_types=['text'])
def reply_hi(message):
  sent_msg = bot.send_message(message.chat.id, iniciar())
  bot.register_next_step_handler(sent_msg, text_response)

def text_response(pm):
  question(pm.text)
  sent_msg = bot.reply_to(pm, response())
  bot.register_next_step_handler(sent_msg, text_response)


bot.polling()


llama_print_timings:        load time =  216200.04 ms
llama_print_timings:      sample time =      17.57 ms /     7 runs   (    2.51 ms per token,   398.41 tokens per second)
llama_print_timings: prompt eval time =  216199.31 ms /   438 tokens (  493.61 ms per token,     2.03 tokens per second)
llama_print_timings:        eval time =    6872.72 ms /     6 runs   ( 1145.45 ms per token,     0.87 tokens per second)
llama_print_timings:       total time =  223173.67 ms /   444 tokens
from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::

Failed to parse function body as JSON schema, falling back to default grammar
'NoneType' object has no attribute 'get'



llama_print_timings:        load time =  216200.04 ms
llama_print_timings:      sample time =      56.67 ms /     2 runs   (   28.33 ms per token,    35.29 tokens per second)
llama_print_timings: prompt eval time =    2851.59 ms /     6 tokens (  475.26 ms per token,     2.10 tokens per second)
llama_print_timings:        eval time =    1092.02 ms /     1 runs   ( 1092.02 ms per token,     0.92 tokens per second)
llama_print_timings:       total time =    4033.76 ms /     7 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =  216200.04 ms
llama_print_timings:      sample time =       2.88 ms /     1 runs   (    2.88 ms per token,   346.86 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    1124.54 ms /     1 runs   ( 1124.54 ms per token,     0.89 tokens per second)
llama_print_timings:       total time =    1143.85 ms /     1 

Print: {'id': 'chatcmpl-8ded8bd2-d9c6-43a0-b2bb-6195a65bd368', 'object': 'chat.completion', 'created': 1717590540, 'model': '/root/.cache/huggingface/hub/models--meetkai--functionary-small-v2.5-GGUF/snapshots/37e7f0678f49a1d360090d0f4b79d138ca2248ee/./functionary-small-v2.5.Q4_0.gguf', 'choices': [{'index': 0, 'logprobs': None, 'message': {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_a1f1TrQHrTQzkqLE8Y4QTUjm', 'type': 'function', 'function': {'name': 'functions.iniciar_pedido\n{}', 'arguments': '{}'}}]}, 'finish_reason': 'tool_calls'}], 'usage': {'prompt_tokens': 450, 'completion_tokens': 7, 'total_tokens': 450}}
iniciar_pedido()


Llama.generate: prefix-match hit

llama_print_timings:        load time =  216200.04 ms
llama_print_timings:      sample time =      52.75 ms /    11 runs   (    4.80 ms per token,   208.55 tokens per second)
llama_print_timings: prompt eval time =   20757.08 ms /    19 tokens ( 1092.48 ms per token,     0.92 tokens per second)
llama_print_timings:        eval time =   22766.38 ms /    10 runs   ( 2276.64 ms per token,     0.44 tokens per second)
llama_print_timings:       total time =   43853.52 ms /    29 tokens
from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_1

Failed to parse function body as JSON schema, falling back to default grammar
'NoneType' object has no attribute 'get'



llama_print_timings:        load time =  216200.04 ms
llama_print_timings:      sample time =      72.27 ms /     2 runs   (   36.13 ms per token,    27.67 tokens per second)
llama_print_timings: prompt eval time =    4007.30 ms /     6 tokens (  667.88 ms per token,     1.50 tokens per second)
llama_print_timings:        eval time =     970.99 ms /     1 runs   (  970.99 ms per token,     1.03 tokens per second)
llama_print_timings:       total time =    5083.69 ms /     7 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =  216200.04 ms
llama_print_timings:      sample time =       2.46 ms /     1 runs   (    2.46 ms per token,   406.83 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    1023.76 ms /     1 runs   ( 1023.76 ms per token,     0.98 tokens per second)
llama_print_timings:       total time =    1046.02 ms /     1 

Print: {'id': 'chatcmpl-3e1ee1e4-22f7-474f-97eb-0f6e9e23f009', 'object': 'chat.completion', 'created': 1717593837, 'model': '/root/.cache/huggingface/hub/models--meetkai--functionary-small-v2.5-GGUF/snapshots/37e7f0678f49a1d360090d0f4b79d138ca2248ee/./functionary-small-v2.5.Q4_0.gguf', 'choices': [{'index': 0, 'logprobs': None, 'message': {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_oT97gGTjxTWpLuGh3CKjToMh', 'type': 'function', 'function': {'name': 'functions.itens_disponiveis_print\n{}', 'arguments': '{}'}}]}, 'finish_reason': 'tool_calls'}], 'usage': {'prompt_tokens': 455, 'completion_tokens': 11, 'total_tokens': 455}}
itens_disponiveis_print()
